## TUTORIAL 21 - Thermal subfin problem
**_Keywords: geometrical parametrization_**

### 1. Introduction
This Tutorial addresses geometrical parametrization for a steady-state heat transfer problem through a fin (solid material with high thermal conductivity). In particular, we will solve a Laplace problem, which deals with the increase of the heat transfer from surfaces by means of a fluid flowing through the fin $\Omega_o(\boldsymbol{\mu})$ which heats it, and divided into four parts, as in the following picture:

<img src="data/thermal_subfin.png" width="70%"/>

We impose uniform heat flux at the base of the primary thermal fin, $\Gamma_{o,1}$, as a model of Joule heating, and we consider Robin boundary conditions on the rest of the fin boundary that model exposition to flowing cooling fluid with zero temperature. On all interior boundaries, $\Gamma_{o,11,12,13}$, we impose the continuity of temperature and heat flux.

This problem is characterized by eight parameters: $\mu_0$ is related to the width of the primary fin; $\mu_1$ takes into account the distance from the base of the primary fin to the subfin; $\mu_2$ is related to the width of the subfin; $\mu_3$ and $\mu_4$ control the length of the right and left portion of the subfin respectively; $\mu_5$ takes into account the isotropic conductivity in the subfin; $\mu_6$ and $\mu_7$ are related to the Biot's coefficient for fin/fluid - subfin/fluid respectively.


The parameter vector $\boldsymbol{\mu}$ is thus given by 
$$
\boldsymbol{\mu} = (\mu_0, \mu_1, \mu_2, \mu_3, \mu_4, \mu_5, \mu_6, \mu_7)
$$
on the parameter domain
$$
\mathbb{P}=[0.1,0.5] \times [0.3,0.6] \times [0.1,0.3] \times [0.2,1]^2 \times [0.1,9]^3.
$$

In order to obtain a faster (yet, provably accurate) approximation of the problem, and avoiding _any_ remeshing, we pursue a model reduction by means of a certified reduced basis reduced order method from a fixed reference domain.

### 2. Parametrized formulation

Let $u_o(\boldsymbol{\mu})$ be the temperature in the domain $\Omega_o(\mu)$.

We will directly provide a weak formulation for this problem: for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u_o(\boldsymbol{\mu})\in\mathbb{V}_o(\boldsymbol{\mu})$ such that

$$a_o\left(u_o(\boldsymbol{\mu}),v_o;\boldsymbol{\mu}\right)=f_o(v_o;\boldsymbol{\mu})\quad \forall v_o\in\mathbb{V}_o(\boldsymbol{\mu})$$

where

* the function space $\mathbb{V}_o(\boldsymbol{\mu})$ is defined as
$$
\mathbb{V}_o(\boldsymbol{\mu}) = \left\{ v \in H^1(\Omega_o(\boldsymbol{\mu})) \right\}
$$
Note that the function space is parameter dependent due to the shape variation. 
* the parametrized bilinear form $a_o(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V}_o(\boldsymbol{\mu}) \times \mathbb{V}_o(\boldsymbol{\mu}) \to \mathbb{R}$ is defined by
$$\begin{align*}
a_o(u_o(\boldsymbol{\mu}), v_o; \boldsymbol{\mu}) & =
\int_{\Omega_{o,1} \cup\,\Omega_{o,2}} \nabla u_o(\boldsymbol{\mu}) \cdot \nabla v_o \ d\boldsymbol{x} + \
\mu_5 \int_{\Omega_{o,3} \cup\,\Omega_{o,4}} \nabla u_o(\boldsymbol{\mu}) \cdot \nabla v_o \ d\boldsymbol{x} \\ & + \
\mu_6 \int_{\Gamma_{o,2,6,10}} u_o(\boldsymbol{\mu}) v_o \ ds + \
\mu_7 \int_{\Gamma_{o,1,3,4,5,7,8,9}} u_o(\boldsymbol{\mu}) v_o \ ds,
\end{align*},$$
* the parametrized linear form $f_o(\cdot; \boldsymbol{\mu}): \mathbb{V}_o(\boldsymbol{\mu}) \to \mathbb{R}$ is defined by
$$f_o(v_o;\boldsymbol{\mu}) = \int_{\Gamma_{o,1}} v_o \ ds.$$

In [ ]:
from dolfin import *
from rbnics import *

## 3. Affine decomposition

In order to obtain an affine decomposition, we recast the problem on a fixed, parameter _independent_, reference domain $\Omega$. As reference domain which choose the one characterized by $\bar{\boldsymbol{\mu}} = (0.3, 0.4, 0.2, 0.5, 0.5, 1, 1, 1)$ which we generate through the generate_mesh notebook provided in the _data_ folder.
Then, we pull back the problem to the reference domain $\Omega$.

In [ ]:
@PullBackFormsToReferenceDomain()
@AffineShapeParametrization("data/subfin1_vertices_mapping.vmp")

class Subfin(EllipticCoerciveProblem):
    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        EllipticCoerciveProblem.__init__(self, V, **kwargs)
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.u = TrialFunction(V)
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)

    # Return custom problem name
    def name(self):
        return "1StageSubfinPOD"

    # Return theta multiplicative terms of the affine expansion of the problem.
    def compute_theta(self, term):
        mu=self.mu
        if term == "a":
            theta_a0 = 1.0
            theta_a1 = mu[5]
            theta_a2 = mu[6]
            theta_a3 = mu[7]
            return (theta_a0, theta_a1, theta_a2, theta_a3)
        elif term == "f":
            theta_f0 = 1.0
            return (theta_f0,)
        elif term == "s":
            theta_s0 = 1.0
            return (theta_s0,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    def assemble_operator(self, term):
        u = self.u
        v = self.v
        dx = self.dx
        ds = self.ds
        if term == "a":
            a0 = inner(grad(u),grad(v))*dx(1)+inner(grad(u),grad(v))*dx(2)+inner(grad(u),grad(v))*dx(3)+inner(grad(u),grad(v))*dx(4)
            a1 = inner(grad(u),grad(v))*dx(5)+inner(grad(u),grad(v))*dx(6)+inner(grad(u),grad(v))*dx(7)+inner(grad(u),grad(v))*dx(8)
            a2 = inner(u,v)*ds(2)+inner(u,v)*ds(6)+inner(u,v)*ds(10)
            a3 = inner(u,v)*ds(1)+inner(u,v)*ds(3)+inner(u,v)*ds(4)+inner(u,v)*ds(5)+inner(u,v)*ds(7)+inner(u,v)*ds(8)+inner(u,v)*ds(9)
            return(a0, a1, a2, a3)
        elif term == "f":
            f0 = v*ds(1)
            return (f0,)
        elif term == "s":
            s0 = v*ds(1)
            return (s0,)
        elif term == "inner_product":
            x0 = inner(grad(u), grad(v))*dx + inner(u, v)*dx
            return(x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")

## 4. Main program
### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh_1.ipynb](data/generate_mesh_1.ipynb) notebook.

In [ ]:
mesh = Mesh("data/subfin1.xml")
subdomains = MeshFunction("size_t", mesh, "data/subfin1_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/subfin1_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P1)

In [ ]:
V = FunctionSpace(mesh, "Lagrange", 1)

### 4.3. Allocate an object of the Subfin class

In [ ]:
problem = Subfin(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [
    (0.1, 0.5),
    (0.3, 0.6),
    (0.1, 0.3),
    (0.2, 1),
    (0.2, 1),
    (0.1, 9),
    (0.1, 9),
    (0.1, 9)
]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with a POD-Galerkin method

In [ ]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(50)
reduction_method.set_tolerance(1e-4)

### 4.5. Perform the offline phase

In [ ]:
reduction_method.initialize_training_set(100)
reduced_problem = reduction_method.offline()

### 4.6. Perform an online solve

In [ ]:
online_mu = (0.32, 0.31, 0.17, 0.51, 0.72, 3.93, 8.03, 3.77)
reduced_problem.set_mu(online_mu)
reduced_solution = reduced_problem.solve()
plot(reduced_solution, reduced_problem=reduced_problem)

### 4.7. Perform an error analysis

In [ ]:
reduction_method.initialize_testing_set(100)
reduction_method.error_analysis(filename="error_analysis")

### 4.8. Perform a speedup analysis

In [ ]:
reduction_method.speedup_analysis(filename="speedup_analysis")

### 5. Assignments


1. Assume now also the conductivity on $\Omega_{o,1}\cup\Omega_{o,2}$ to be paramerized, i.e.\
$$
\kappa(\mu_5, \mu_8) =
\begin{cases}
\mu_8 & \text{in } \Omega_{o,1}\cup\Omega_{o,2},\\
\mu_5 & \text{in } \Omega_{o,3}\cup\Omega_{o,4},\\
\end{cases}, \quad \text{for} \quad
\boldsymbol{\mu} = (\mu_5, \mu_8)\in[0.1,9]^2.
$$
Repeat the worked out problem in this case.

2. Use a RB method instead of a POD-Galerkin, employing the Successive Constraint Method (SCM) in the computation of a lower bound of the coercivity constant. Compare the results of the error analysis and speedup analysis to the ones obtained here.